In [ ]:
# @title

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# Load the dataset
df = pd.read_csv('data.csv')  # Replace 'your_dataset.csv' with your actual dataset file

# Assuming 'Time' is the column containing the time values in "mm:ss.s" format
df['Timestamp'] = pd.to_datetime(df['Date\t'] + ' ' + df['Time'], format="%d-%m-%Y %M:%S.%f")
df.set_index('Timestamp', inplace=True)
df.drop(['Date\t', 'Time'], axis=1, inplace=True)

In [ ]:
# Normalize numerical features
scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# Generator model
generator = Sequential([
    Dense(16, activation='relu', input_shape=(9,)),
    Dense(9, activation='sigmoid')
])

# Discriminator model
discriminator = Sequential([
    Dense(16, activation='relu', input_shape=(9,)),
    Dense(1, activation='sigmoid')
])

# Combined model (GAN)
discriminator.trainable = False
gan_input = Input(shape=(9,))
x = generator(gan_input)
gan_output = discriminator(x)
gan = Model(gan_input, gan_output)
gan.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')

# Compile discriminator
discriminator.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')

# Compile GAN
gan.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')

# Training the GAN
epochs = 5000
batch_size = 32

for epoch in range(epochs):
    idx = np.random.randint(0, df_normalized.shape[0], batch_size)
    real_data = df_normalized.iloc[idx]

    noise = np.random.normal(0, 1, size=[batch_size, 9])
    generated_data = generator.predict(noise)

    X = np.concatenate([real_data, generated_data])
    y_dis = np.zeros(2 * batch_size)
    y_dis[:batch_size] = 0.9

    d_loss = discriminator.train_on_batch(X, y_dis)

    noise = np.random.normal(0, 1, size=[batch_size, 9])
    y_gen = np.ones(batch_size)

    g_loss = gan.train_on_batch(noise, y_gen)

    if epoch % 100 == 0:
        print(f"Epoch {epoch} | Discriminator Loss: {d_loss} | Generator Loss: {g_loss}")

# Generate synthetic normal data
noise = np.random.normal(0, 1, size=[df_normalized.shape[0], 9])
synthetic_normal_data = generator.predict(noise)

# Calculate BCE loss between actual and synthetic normal data
bce_loss = np.mean(np.abs(df_normalized - synthetic_normal_data), axis=1)

# Adjust anomaly threshold based on your experimentation
anomaly_threshold = 0.7  # You can try different values



In [ ]:

anomalies = df.reset_index()[bce_loss > anomaly_threshold]

# Get timestamps of anomalies
anomaly_timestamps = anomalies['Timestamp']


print("Anomaly Timestamps:")
print(anomaly_timestamps)
